# Using oemof.tabular.facades

This ipython-notebook is designed to describe the usage and functionality of the facades that are based on the [oemof.solph](https://oemof.readthedocs.io/en/stable/oemof_solph.html) package. If you are scripting and writing your own model you can easily use the classes provided by solph. The main potential of the facades is to provide 
an easy interface when defining the energy system in a input datapackage or any other tabular source. To see how you can do this have a look at the `model-from-tabular-data.ipynb` example. 


**NOTE**: Numeric values in this example are not representative for any energy system and just randomly selected. Also this model is not necessarily a feasible optimization problem. It is **only** designed for illustration of class usage. 



Author:

Simon.Hilpert (@uni-flensburg.de), Europa Universitaet Flensburg, March 2019

### Python Imports

In [ ]:
import pandas as pd

from oemof.solph import EnergySystem, Model, Bus
import oemof.tabular.facades as fc

# for simplicity we just use 3 timesteps
es = EnergySystem(timeindex=pd.date_range('2018', periods=3, freq='H'))

### Bus

First we will create the required buses for this example. As these objects will be used when instantiating the components, we assign them to python variables and add these to the energy system object `es` using ist `.add()` method. The `balanced` (default: True) argument can be used to relax the energy balance for a bus, i.e. all inputs do not have to sum with all output flows to zero any longer.  

In [ ]:
elec_bus = Bus(label="elec_bus")
elec_bus_1 = Bus(label="elec_bus_1")
heat_bus = Bus(label="heat_bus")
fuel_bus = Bus(label="fuel_bus", balanced=True)

es.add(elec_bus, elec_bus_1, heat_bus, fuel_bus)

### Volatile

This class can be used to model PV oder Wind power plants. The equations for this component are:

$$x_{wind}^{flow} = c_{wind}^{capacity} \cdot c_{wind}^{profile}(t) \qquad \forall t \in T$$

Where $x_{wind}^{flow}$ denotes the production (endogenous variable) of the volatile object to the bus. 


In [ ]:
es.add(
    fc.Volatile(
        label="wind",
        carrier="wind",
        tech="onshore",
        capacity=150,
        bus=elec_bus,
        profile=[0.2, 0.3, 0.25],
    )
)

#### Volatile component investment 
If the investment mode is used, i.e. `capacity_cost` attribute not None and `capacity` attribute set to None 
the right hand side of the equation changes as the exogenous variable $c^{capacity}$ is now replaced by an endognous variable.  

$$x^{f,ow}_{wind} \leq x^{capacity}_{wind} \cdot c^{profile}_{wind}(t) \qquad \forall t \in T$$

In [ ]:
es.add(
    fc.Volatile(
        label="wind_invest",
        carrier="wind",
        tech="onshore",
        capacity_cost=200,
        exapandable=True,
        bus=elec_bus,
        profile=[0.2, 0.3, 0.25],
    )
)

### Dispatchable

The `Dispatchble` component works very similar to the volatile component. The only difference here is the $\leq$ 
sign in the constraint, which allows to dispatch the power within the limit of the lower and upper bounds.

$$x_{flow}^{elec\_bus} \leq c_{ccgt}^{capacity} \cdot c_{ccgt}^{profile}(t) \qquad \forall t \in T$$

**NOTE**: 

You can also set the parameters for the output of this component by using the argument `output_parameters`. To see all options see: [oemof Flow](https://oemof.readthedocs.io/en/stable/api/oemof.solph.html#oemof.solph.network.Flow)

In [ ]:
es.add(
    fc.Dispatchable(
        bus=elec_bus,
        label="ccgt",
        carrier="gas",
        tech="ccgt",
        capacity=100,
        marginal_cost=25,
        output_parameters={
            'summed_min': 1000,
            'summed_max': 2000}
    )
)

### Reservoir

The reservoir component inherit from the [GenericStorage](https://oemof.readthedocs.io/en/stable/api/oemof.solph.html#oemof.solph.components.GenericStorage). However the input of the reservoir is not taken from the bus but defined as an absolute inflow to the reservoir from a source. This source object is created under the hood interanally when a `Reservoir` object is instantiated. 

In [ ]:
es.add(
    fc.Reservoir(
        bus=elec_bus,
        label="rsv",
        carrier="hydro",
        tech="reservoir",
        capacity=150,
        storage_capacity=1500,
        efficiency=0.9,
        profile=[10, 5, 3],
        initial_storage_level=1, # oemof.solph arguments
        balanced=False # oemof.solph argument
    )
)


### Storage

The Storage component is based on the [GenericStorage](https://oemof.readthedocs.io/en/stable/api/oemof.solph.html#oemof.solph.components.GenericStorage) class of `oemof.solph`. Therefore you may use all arguments that exist for the parent class in addition to the ones defined for the component itself.

In [ ]:
es.add(
    fc.Storage(
        label="storage",
        bus=elec_bus,
        carrier="lithium",
        tech="battery",
        capacity_cost=10,
        expandable=True,
        invest_relation_output_capacity=1/6, # oemof.solph
        marginal_cost=5,
        balanced=True, # oemof.solph argument
        initial_storage_level=1, # oemof.solph argument
        max_storage_level=[0.9, 0.95, 0.8], # oemof.solph argument
    )
)

### Extraction and Backpressure Turbine 

The extraction turbine facade directly inherit from the [ExtractionTurbineCHP](https://oemof.readthedocs.io/en/stable/api/oemof.solph.html#oemof.solph.components.ExtractionTurbineCHP) class where as the backpressure facade directly inherit from the [Transformer](https://oemof.readthedocs.io/en/stable/api/oemof.solph.html#oemof.solph.network.Transformer) class.

Both components may also be used in the investment mode by setting the capacity costs. The capacity cost are related to the electrical output of the component, i.e. Euro/MWhel.


In [ ]:
es.add(
    fc.ExtractionTurbine(
        label="ext",
        electricity_bus=elec_bus,
        heat_bus=heat_bus,
        fuel_bus=fuel_bus,
        carrier='gas',
        tech='ext',
        capacity=10,
        condensing_efficiency=0.5,
        electric_efficiency=0.4,
        thermal_efficiency=0.3
    )
)

es.add(
    fc.BackpressureTurbine(
        label="bp",
        electricity_bus=elec_bus,
        heat_bus=heat_bus,
        fuel_bus=fuel_bus,
        carrier='gas',
        tech='bp',
        capacity=10,
        electric_efficiency=0.4,
        thermal_efficiency=0.3
    )
)

### Conversion 

The conversion component is a simplified interface to the [Transformer](https://oemof.readthedocs.io/en/stable/api/oemof.solph.html#oemof.solph.network.Transformer) class with the **restriction of 1 input and 1 output**. 

In [ ]:
es.add(
    fc.Conversion(
        label='pth',
        from_bus=elec_bus,
        to_bus=heat_bus,
        carrier='electricity',
        tech='hp',
        capacity=10,
        capacity_cost=54,
        expandable=True,
        capacity_potential=20,
        efficiency=0.9,
        thermal_efficiency=0.3
    )
)

### Commodity

A commodity can be used to model a limited source for the complete time horizon of the problem.

$$\sum_t x^{flow}_{fuel}(t) \leq c^{amount}_{fuel} $$

In [ ]:
es.add(
    fc.Commodity(
        label='fuel',
        bus=fuel_bus,
        amount=1000000,
        carrier='fuel',
        tech='commodity',
        marginal_cost=0.5
    )
)

### Link

This component also just provides an simpliefied interface for the [Link](https://oemof.readthedocs.io/en/stable/api/oemof.solph.html#oemof.solph.custom.Link) of the oemof solph package. 

In [ ]:
es.add(
    fc.Link(
        label='link',
        from_bus=elec_bus,
        to_bus=elec_bus_1,
        loss=0.05,
        capacity=100
    )
)

### Load 

The load is similar to the `Volatile` component, except that it acts as a sink (1-input). Therefore you may also use the argument `input_parameters` to adapt its behaviour. 

In [ ]:
es.add(
    fc.Load(
        label="elec_load", 
        bus=elec_bus, 
        amount=500e3, 
        profile=[0.4, 0.1, 0.5]))

es.add(
    fc.Load(
        label="heat_load", 
        bus=heat_bus, 
        amount=200e3, 
        profile=[0.1, 0.23, 0.7]))

### Adding Other oemof.solph Components

As the energy system and all components are solph based objects you may add any other oemof.solph object to your energy system. This is straight forward when you are scripting. The full levearge of the facades is however gained when using the datapackage reader. For this datapackage reader **only** facades are working in a proper way. 

### Create Model and Inspect

In [ ]:
m = Model(es)

# uncommet to get lp-file (path will be of this file)
# m.write(io_options={'symbolic_solver_labels': True})


In [ ]:
m.InvestmentFlow.pprint()

In [ ]:
m.Flow.pprint()